### FINA 4380 with Marius Popescu

### Rolling Portfolio Optimization

In [2]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from scipy.optimize import minimize

In [3]:
start = dt.datetime(2020, 1, 1)
end = dt.datetime.now()

rf_rate = web.DataReader('F-F_Research_Data_Factors_daily','famafrench', start, end)[0][['RF']].div(100)
rf_rate.tail()

,RF
Date,
2023-02-22,0.00018
2023-02-23,0.00018
2023-02-24,0.00018
2023-02-27,0.00018
2023-02-28,0.00018


In [4]:
ticker_list= ['VDE','VAW', 'VIS', 'VCR', 'VDC','VHT', 'VFH', 'VGT', 'VOX', 'VPU', 'VNQ']

returns = yf.download(ticker_list,start-pd.offsets.BDay(1),end+pd.offsets.BDay(1))['Adj Close'].pct_change().dropna()
returns.tail()

[*********************100%***********************]  11 of 11 completed


,VAW,VCR,VDC,VDE,VFH,VGT,VHT,VIS,VNQ,VOX,VPU
Date,,,,,,,,,,,
2023-03-29,0.013085,0.017827,0.005670,0.014420,0.014704,0.020435,0.004107,0.013191,0.022163,0.013537,0.014052
2023-03-30,0.005568,0.008389,0.002923,0.002649,-0.003623,0.010093,0.004346,0.002828,0.013084,0.003922,0.004047
2023-03-31,0.015128,0.024918,0.007235,0.005900,0.011427,0.016240,0.011710,0.013675,0.021402,0.020695,0.007926
2023-04-03,0.005736,-0.006573,0.006201,0.047711,-0.001797,-0.001323,0.009729,0.002782,-0.007948,0.002690,-0.007660
2023-04-04,-0.016272,-0.002112,-0.004622,-0.019552,-0.012219,-0.006338,-0.000581,-0.024969,-0.001335,0.000619,0.004577


In [5]:
returns=returns[returns.index<=rf_rate.index[-1]]
returns.tail(1)

,VAW,VCR,VDC,VDE,VFH,VGT,VHT,VIS,VNQ,VOX,VPU
Date,,,,,,,,,,,
2023-02-28,0.005154,0.000445,-0.008431,-0.014537,0.002065,-0.00105,-0.006895,-0.001873,-0.001398,-0.000441,-0.016453


In [6]:
len(returns.index)

795

In [7]:
# Window for the optimization is 500 trading days

We want to find the optimal portfolio over each of the following groups of observations:
* observations 1 - 500
* observations 2 - 501
* observations 3 - 502
* observations 4 - 503
* ...
* observations 296 - 795

In [9]:
#for i in range(0,len(returns.index)-500+1):
#    print(i,i+500-1)

In [20]:
optimal_weights = pd.DataFrame()

for i in range(0,len(returns.index)-500+1):
    # Define the excess portfolio returns function
    def ex_port_ret(weights):
        ex_port_ret = (np.dot(returns.iloc[i:i+500],weights) - rf_rate.iloc[i:i+500]['RF']).mean()*250
        return ex_port_ret

    # Define the excess portfolio standard deviation function
    def ex_port_std(weights):
        ex_port_std = (np.dot(returns.iloc[i:i+500],weights) - rf_rate.iloc[i:i+500]['RF']).std()*np.sqrt(250)
        return ex_port_std

    # Define the negative Sharpe Ratio function that we will minimize
    def neg_SR(weights):
        SR = ex_port_ret(weights) / ex_port_std(weights)
        return (-1)*SR

    # By convention of minimize function it should be a function
    # that returns zero for conditions
    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

    # Weights must be between 0 and 1
    boundaries=[(0,1)]
    bounds = tuple(boundaries * len(returns.columns))

    # Initial Guess (equally weighted)
    init_guess = np.full(len(returns.columns),1/len(returns.columns))
    
    optimal_port = minimize(neg_SR,init_guess,bounds = bounds,constraints=constraints)
    
    optimal_weights = pd.concat([optimal_weights,
                                pd.DataFrame(optimal_port.x.reshape(1,len(ticker_list)).round(4),
                                             columns=[ticker_list],
                                             index=[returns.iloc[i:i+500].index[-1]])],
                                axis=0)

In [21]:
optimal_weights.head()

,VDE,VAW,VIS,VCR,VDC,VHT,VFH,VGT,VOX,VPU,VNQ
2021-12-23,0.0,0.6913,0.0,0.0,0.0,0.3087,0.0,0.0,0.0,0.0,0.0
2021-12-27,0.0,0.6626,0.0,0.0,0.0,0.3374,0.0,0.0,0.0,0.0,0.0
2021-12-28,0.0,0.6862,0.0,0.0,0.0,0.3138,0.0,0.0,0.0,0.0,0.0
2021-12-29,0.0,0.7062,0.0,0.0,0.0,0.2938,0.0,0.0,0.0,0.0,0.0
2021-12-30,0.0,0.7172,0.0,0.0,0.0,0.2828,0.0,0.0,0.0,0.0,0.0


In [22]:
optimal_weights.tail()

,VDE,VAW,VIS,VCR,VDC,VHT,VFH,VGT,VOX,VPU,VNQ
2023-02-22,0.0,0.0,0.3035,0.5530,0.0,0.0,0.0,0.0,0.0,0.0,0.1435
2023-02-23,0.0,0.0,0.3512,0.5842,0.0,0.0,0.0,0.0,0.0,0.0,0.0647
2023-02-24,0.0,0.0,0.2869,0.6023,0.0,0.0,0.0,0.0,0.0,0.0,0.1108
2023-02-27,0.0,0.0,0.2984,0.5836,0.0,0.0,0.0,0.0,0.0,0.0,0.1180
2023-02-28,0.0,0.0,0.3787,0.5735,0.0,0.0,0.0,0.0,0.0,0.0,0.0478


In [23]:
len(optimal_weights.index)

296

In [25]:
#optimal_weights.iloc[0]

In [27]:
#returns.iloc[[0+500]]

In [28]:
np.dot(optimal_weights.iloc[0],returns.iloc[0+500])

0.012837458449152872

In [14]:
#optimal_weights.iloc[1]

In [15]:
#returns.iloc[[1+500]]

In [29]:
np.dot(optimal_weights.iloc[1],returns.iloc[1+500])

-0.0021463358146715083

In [30]:
port_returns = pd.DataFrame()
for i in range(0,len(optimal_weights)-1):
    port_returns = pd.concat([port_returns,
                              pd.DataFrame({'Roll_OP':np.dot(optimal_weights.iloc[i],returns.iloc[i+500])},
#                              pd.DataFrame({'NoRoll_OP':np.dot(optimal_weights.iloc[0],returns.iloc[i+500])})],
                                           index = [returns.index[i+500]])],
                                           axis=0)

In [32]:
port_returns.head(2)

,Roll_OP
2021-12-27,0.012837
2021-12-28,-0.002146


In [34]:
port_returns = pd.concat([port_returns,
                          pd.DataFrame({'NoRoll_OP':np.dot(returns.iloc[0+500:], optimal_weights.iloc[0])},index = returns.index[0+500:])],
                         axis=1)

In [35]:
port_returns.head(2)

,Roll_OP,NoRoll_OP
2021-12-27,0.012837,0.012837
2021-12-28,-0.002146,-0.001936


In [36]:
port_returns.describe()

,Roll_OP,NoRoll_OP
count,295.000000,295.000000
mean,-0.000388,-0.000759
std,0.016031,0.020360
min,-0.051031,-0.058245
25%,-0.009442,-0.014532
50%,0.000096,-0.001234
75%,0.010172,0.014218
max,0.052318,0.078362
